<a href="https://colab.research.google.com/github/jbsdoki/Squishy_Robots_Quant_Models/blob/main/Squish_Robot_Quant_Model_5_multi_modal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model Description

This model was produced by and for Squishy Robotics for the task of identifying and classifying methane leaks.


This model was made in conjunction with a synthetic dataset of 2 channel, 240 by 320 greyscale images of methane leaks
(2 x 240 x 320)
The first channel is a greyscale background image and the second channel is a greyscale gas plume image.



This model is experimental and uses the Optuna Hyperparameter Optimizer to search for successful hyperparameters (Learning Rate, Optimizer, Batch Size, Dropout %, etc...) and different optimizers. As such if you want to test a specific Model architecture you need to comment out the Optuna code and run a train/test on that specific model.

In [2]:
pip install optuna #Hyperparameter Optimizer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 32.6 MB/s eta 0:00:00


In [3]:
import os
import numpy as np

from collections import defaultdict
from collections import Counter

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from torch.utils.data import random_split

# Hyperparameter Search
import optuna

import json
import glob


In [4]:
# This may take several minutes, the synthetic dataset can be large
!unzip -q Final_Dataset.zip

## Print out the structure of the data

In [5]:
# Loading an example file to demonstrate the dimensions
# This file might not exist, change the name to one that does to show the
# dimensions
file_path = './Final_Dataset/data/class_0/1237_frame_1004_class_0.npy'
sample_data = np.load(file_path)
print(f"Shape of preprocessed sample data: {sample_data.shape}")
print(f"Data type of preprocessed sample data: {sample_data.dtype}")

# GasVid synthetic processed dataset should be 2 channels, 240x320 in dimension

Shape of preprocessed sample data: (2, 240, 320)
Data type of preprocessed sample data: float32


In [7]:
# Assuming the data is in 'Final_Dataset/data' and class folders are named 'class_0' ... 'class_7'
data_dir = 'Final_Dataset/data'
classes = sorted(os.listdir(data_dir))
print(f"Classes: {classes}")

Classes: ['class_0', 'class_1', 'class_2', 'class_3', 'class_4', 'class_5', 'class_6', 'class_7']


## Create a dataset and dataloader

In [12]:
class Multi_Modal_Dataset(Dataset):
    def __init__(self, numpy_files, json_files, labels):
        """
        numpy_dir points to all the numpy 2 channel frames that were collected
          from METEC. This is designed to be 1st Channel Greyscale image of
          background, 2nd channel is just the gas plume scaled to some ppm
        json_dir points to all the metadata (ppm, distance, etc) that was
          collected from METEC or estimated using BEST Labs algorithms
        """
        self.numpy_files = numpy_files
        self.json_files = json_files
        self.labels = labels


    def __len__(self):
      return len(self.numpy_files)


    def __getitem__(self, idx):
      numpy_path = self.numpy_files[idx]
      image_data = np.load(numpy_path)
      image_tensor = torch.from_numpy(image_data).float()

      json_path = self.json_files[idx]
      with open(json_path, 'r') as f:
        metadata = json.load(f)

      metadat_features = self._extract_metadata_features(metadata)
      metadata_tensor = torch.tensor(metadat_features, dtype=torch.float32)

      label = self.labels[idx]

      return image_tensor, metadata_tensor, label


    def _extract_metadata_features(self, metadata):
      """
      Extracts a few entries from the metadata.
        For now:
          distance
          ppm
        In the future
          windspeed
          angle?
      """

      features = []

      # If the features exist, extract them, else place 0.0
      # Print warning statements if unable to retrieve the data
      distance = metadata.get("distance_m", None)
      if distance is None or distance == 0.0:
          print(f"WARNING: Invalid or missing distance_m value: {distance}")
          print(f"  Metadata keys available: {list(metadata.keys())}")
          features.append(0.0)
      else:
          features.append(distance)

      ppm = metadata.get("ppm", None)
      if ppm is None:
          print(f"WARNING: Missing ppm value")
          print(f"  Metadata keys available: {list(metadata.keys())}")
          features.append(0.0)
      else:
          features.append(ppm)

      return features

In [13]:
numpy_dir = "./Final_Dataset/data"
json_dir = "./Final_Dataset/metadata"

all_numpy_files = []
all_json_files = []
all_labels = []

print(f"Looking in: {numpy_dir}")
print(f"Directory exists: {os.path.exists(numpy_dir)}\n")

# Load each class separatley, collect the numpy and json files for a certain
# class at the same time
for class_idx in range(8):
    numpy_class_dir = os.path.join(numpy_dir, f"class_{class_idx}")
    json_class_dir = os.path.join(json_dir, f"class_{class_idx}")

    numpy_files_in_class = sorted(glob.glob(os.path.join(numpy_class_dir, "*.npy")))

    print(f"Class {class_idx}: Found {len(numpy_files_in_class)} files")

    for numpy_file in numpy_files_in_class:
        base_name = os.path.splitext(os.path.basename(numpy_file))[0]
        video_id = base_name.split('_')[0]


        json_filename = f"{video_id}_class_{class_idx}.json"
        json_file = os.path.join(json_class_dir, json_filename)

        if os.path.exists(json_file):
            all_numpy_files.append(numpy_file)
            all_json_files.append(json_file)
            all_labels.append(class_idx)
        else:
            print(f"WARNING: JSON missing for {base_name}")

print(f"\n{'='*60}")
print(f"TOTAL: {len(all_numpy_files)} numpy files")
print(f"TOTAL: {len(all_json_files)} json files")
print(f"{'='*60}\n")

# Only continue if we have files
if len(all_numpy_files) == 0:
    raise ValueError("No files found! Check your paths above.")

# Now continue with video splitting
video_to_indices = defaultdict(list)
for idx, numpy_file in enumerate(all_numpy_files):
    video_id = os.path.basename(numpy_file).split('_')[0]
    video_to_indices[video_id].append(idx)

print(f"Number of unique videos: {len(video_to_indices)}")
print(f"Video IDs: {sorted(video_to_indices.keys())}\n")

# ... rest of your code

Looking in: ./Final_Dataset/data
Directory exists: True

Class 0: Found 5395 files
Class 1: Found 5393 files
Class 2: Found 5393 files
Class 3: Found 5397 files
Class 4: Found 5382 files
Class 5: Found 5411 files
Class 6: Found 5380 files
Class 7: Found 5406 files

TOTAL: 43157 numpy files
TOTAL: 43157 json files

Number of unique videos: 28
Video IDs: ['1237', '1238', '1239', '1240', '1241', '1242', '1467', '1468', '1469', '1470', '1471', '1472', '2559', '2560', '2561', '2562', '2563', '2564', '2566', '2567', '2568', '2569', '2571', '2578', '2579', '2580', '2581', '2583']



In [14]:
video_to_indices = defaultdict(list) #Make an empty dictionary of lists

for idx, numpy_file in enumerate(all_numpy_files):
    video_id = os.path.basename(numpy_file).split('_')[0] #Extract 4 digit code from numpy filename
    video_to_indices[video_id].append(idx)

video_ids = list(video_to_indices.keys())

# Split the video into train and test
train_vids, test_vids = train_test_split(video_ids, test_size=0.2, random_state=42)

# Verify no overlap
overlap = set(train_vids) & set(test_vids)
if overlap:
    print(f"\nVideos overlap: {overlap}")
else:
    print(f"\nNo video overlap - train and test are separate")

train_indices = []
test_indices = []

for vid in train_vids:
    train_indices.extend(video_to_indices[vid])
for vid in test_vids:
    test_indices.extend(video_to_indices[vid])

# Create file lists
train_numpy = [all_numpy_files[i] for i in train_indices]
train_json = [all_json_files[i] for i in train_indices]
train_labels_list = [all_labels[i] for i in train_indices]

test_numpy = [all_numpy_files[i] for i in test_indices]
test_json = [all_json_files[i] for i in test_indices]
test_labels_list = [all_labels[i] for i in test_indices]


Video ID's:  ['1237', '1238', '1239', '1240', '1241', '1242', '1467', '1468', '1469', '1470', '1471', '1472', '2559', '2560', '2561', '2562', '2563', '2564', '2566', '2567', '2568', '2569', '2571', '2578', '2579', '2580', '2581', '2583']

No video overlap - train and test are separate


In [16]:
# SHOW FINAL SPLIT STATISTICS
print(f"\n{'='*60}")
print("DATASET STATISTICS")
print("="*90)

print(f"\nTRAINING SET:")
print(f"   Total samples: {len(train_numpy)}")
print(f"   From {len(train_vids)} videos: {sorted(train_vids)}")

# Count samples per class in training
train_class_counts = Counter(train_labels_list)
print(f"\n   Samples per class:")
for class_id in range(8):
    count = train_class_counts.get(class_id, 0)
    percentage = (count / len(train_numpy) * 100) if len(train_numpy) > 0 else 0
    print(f"      Class {class_id}: {count:5d} samples ({percentage:5.2f}%)")

print(f"\nTEST SET:")
print(f"   Total samples: {len(test_numpy)}")
print(f"   From {len(test_vids)} videos: {sorted(test_vids)}")

# Count samples per class in testing
test_class_counts = Counter(test_labels_list)
print(f"\n   Samples per class:")
for class_id in range(8):
    count = test_class_counts.get(class_id, 0)
    percentage = (count / len(test_numpy) * 100) if len(test_numpy) > 0 else 0
    print(f"      Class {class_id}: {count:5d} samples ({percentage:5.2f}%)")

# VERIFY ALL CLASSES PRESENT
print(f"\n{'='*70}")
print("VERIFICATION")
print("="*70)

train_classes = set(train_labels_list)
test_classes = set(test_labels_list)
missing_train = set(range(8)) - train_classes
missing_test = set(range(8)) - test_classes

if missing_train:
    print(f"WARNING: Training missing classes {missing_train}")
else:
    print(f"Training set has all 8 classes")

if missing_test:
    print(f"WARNING: Testing missing classes {missing_test}")
else:
    print(f"Test set has all 8 classes")

# Show train/test split ratio
total_samples = len(train_numpy) + len(test_numpy)
train_ratio = len(train_numpy) / total_samples * 100
test_ratio = len(test_numpy) / total_samples * 100
print(f"\nSplit ratio: {train_ratio:.1f}% train / {test_ratio:.1f}% test")

print(f"\n{'='*70}")
print("DATA SPLIT COMPLETE AND VERIFIED")
print("="*70)



DATASET STATISTICS

TRAINING SET:
   Total samples: 33903
   From 22 videos: ['1238', '1239', '1240', '1241', '1242', '1467', '1468', '1471', '1472', '2560', '2561', '2562', '2563', '2564', '2566', '2567', '2568', '2571', '2578', '2579', '2581', '2583']

   Samples per class:
      Class 0:  4232 samples (12.48%)
      Class 1:  4232 samples (12.48%)
      Class 2:  4242 samples (12.51%)
      Class 3:  4243 samples (12.52%)
      Class 4:  4232 samples (12.48%)
      Class 5:  4250 samples (12.54%)
      Class 6:  4229 samples (12.47%)
      Class 7:  4243 samples (12.52%)

TEST SET:
   Total samples: 9254
   From 6 videos: ['1237', '1469', '1470', '2559', '2569', '2580']

   Samples per class:
      Class 0:  1163 samples (12.57%)
      Class 1:  1161 samples (12.55%)
      Class 2:  1151 samples (12.44%)
      Class 3:  1154 samples (12.47%)
      Class 4:  1150 samples (12.43%)
      Class 5:  1161 samples (12.55%)
      Class 6:  1151 samples (12.44%)
      Class 7:  1163 samples

In [17]:
train_dataset = Multi_Modal_Dataset(train_numpy, train_json, train_labels_list)
test_dataset = Multi_Modal_Dataset(test_numpy, test_json, test_labels_list)

# Define the CNN model

## Define the Optuna Objective Function

This function will be called by Optuna for each trial. It will:
1. Suggest hyperparameters using the trial object.
2. Build and train the CNN model with the suggested hyperparameters.
3. Evaluate the model on a validation set
4. Return the metric to minimize (loss) or maximize (accuracy).

In [23]:
def objective(trial):

    #############################
    # All Hyperparameters Tested
    #############################
    lr = trial.suggest_float('lr', 1e-5, 1e-1, log=True)
    optimizer_name = trial.suggest_categorical('optimizer', ['Adam', 'SGD', 'AdamW'])
    momentum = trial.suggest_float('momentum', 0.0, 0.99) if optimizer_name in ['SGD'] else 0.0
    weight_decay = trial.suggest_float('weight_decay', 0.0, 0.01)
    hidden_size = trial.suggest_int('hidden_size', 64, 256)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])
    num_epochs = trial.suggest_int('num_epochs', 5, 10)
    fc_drop_rate = trial.suggest_float('fc_drop_rate', 0.2, 0.6)
    cnn_drop_rate = trial.suggest_float('cnn_drop_rate', 0.0, 0.3)

    #####################
    # Define the Model
    #####################
    class VideoGasNet(nn.Module):
        def __init__(self, num_metadata_feats = 2, fc_drop_rate = 0.3, cnn_drop_rate = 0.3):
            super(VideoGasNet, self).__init__()

            self.conv1    = nn.Conv2d(2, 32, kernel_size=3, padding=1)
            self.bn1      = nn.BatchNorm2d(32)
            self.relu1    = nn.ReLU()
            self.pool1    = nn.MaxPool2d(kernel_size=2, stride=2)
            self.dropout1 = nn.Dropout2d(cnn_drop_rate)

            self.conv2    = nn.Conv2d(32, 64, kernel_size=3, padding=1)
            self.bn2      = nn.BatchNorm2d(64)
            self.relu2    = nn.ReLU()
            self.pool2    = nn.MaxPool2d(kernel_size=2, stride=2)
            self.dropout2 = nn.Dropout2d(cnn_drop_rate)

            self.conv3    = nn.Conv2d(64, 128, kernel_size=3, padding=1)
            self.bn3      = nn.BatchNorm2d(128)
            self.relu3    = nn.ReLU()
            self.pool3    = nn.MaxPool2d(kernel_size=2, stride=2)
            self.dropout3 = nn.Dropout2d(cnn_drop_rate)

            # Original VGN had 4 blocks, performance seems to drop with additional
            # blocks, testing current architecture before uncommenting this
            # self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
            # self.relu4 = nn.ReLU()
            # self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)

            # Calculate flatten size from conv layers from input(240x320)
            # flatten_size = 128 * (240 // 8) * (320 // 8)
            # 2^3 = 8 use for every conv + relu + pool block
            # If adding more blocks multiply another 2 (2^4 = 16 for for blocks)
            cnn_flatten_size = 128 * (240 // 8) * (320 // 8)

            self.metadata_fc1 = nn.Linear(num_metadata_feats, 64)
            self.metadata_bn1 = nn.BatchNorm1d(64)
            self.metadata_relu1 = nn.ReLU()
            self.metadata_dropout = nn.Dropout(fc_drop_rate)

            # Append the metadata to the fully connected layer
            combined_size = cnn_flatten_size + 64

            self.fc1 = nn.Linear(combined_size, hidden_size)
            self.bn4 = nn.BatchNorm1d(hidden_size)
            self.relu4 = nn.ReLU()
            self.dropout4 = nn.Dropout(fc_drop_rate)
            self.fc2 = nn.Linear(hidden_size, 8)

        def forward(self, image, metadata):
            # Convolutional Blocks
            x = self.dropout1(self.pool1(self.relu1(self.bn1(self.conv1(image)))))
            x = self.dropout2(self.pool2(self.relu2(self.bn2(self.conv2(x)))))
            x = self.dropout3(self.pool3(self.relu3(self.bn3(self.conv3(x)))))

            x = x.view(x.size(0), -1)

            # Metadata from json blocks
            meta = self.metadata_relu1(self.metadata_bn1(self.metadata_fc1(metadata)))
            meta = self.metadata_dropout(meta)

            # concatenate and flatten
            combined = torch.cat([x, meta], dim=1)

            # Fully Connected Blocks (Neural Network)
            combined = self.relu4(self.bn4(self.fc1(combined)))
            combined = self.dropout4(combined)
            output = self.fc2(combined)

            return output

    model = VideoGasNet(num_metadata_feats = 2, fc_drop_rate=fc_drop_rate, cnn_drop_rate=cnn_drop_rate)

    ###############################
    # Define optimizer
    ###############################
    if optimizer_name == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
    elif optimizer_name == 'RMSprop':
        optimizer = optim.RMSprop(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
    elif optimizer_name == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    elif optimizer_name == 'AdamW':
        optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    elif optimizer_name == 'Adadelta':
        optimizer = optim.Adadelta(model.parameters(), lr=lr, weight_decay=weight_decay)
    elif optimizer_name == "Muon":
        optimizer = optim.Muon(model.parameters(), lr=lr, weight_decay=weight_decay)
    else:
        raise ValueError(f"Unknown optimizer name: {optimizer_name}")

    criterion = nn.CrossEntropyLoss()

    ##########################################
    # Create DataLoaders with trial batch_size
    ##########################################
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    ###############################
    # Train the model
    ###############################
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)


    print(f"\n{'='*70}")
    print(f"Trial {trial.number} | lr={lr:.6f} | optimizer={optimizer_name} | "
          f"batch={batch_size} | hidden={hidden_size}")
    print(f"{'='*70}")


    model.train()
    train_correct = 0
    train_total = 0
    for epoch in range(num_epochs):
        train_correct = 0
        train_total = 0

        for images, metadata, labels in train_loader:
            images = images.to(device)
            metadata = metadata.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(images, metadata)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # Calculate training accuracy
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()

        #Print out training during each epoch
        train_accuracy = train_correct / train_total

        # Print training accuracy for this epoch
        print(f"Epoch [{epoch+1:2d}/{num_epochs} ] Train Acc: {train_accuracy:.4f}")

    #####################
    # Evaluate the model
    #####################
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():

        for images, metadata, labels in test_loader:
            images = images.to(device)
            metadata = metadata.to(device)
            labels = labels.to(device)

            outputs = model(images, metadata)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total

    print(f"Validation Acc: {accuracy:.4f}")
    print(f"{'='*70}\n")

    return accuracy


## Run the Optuna Study

Now we will create an Optuna study and run the optimization process.

In [24]:
# Create a study object and specify the direction of optimization (maximize accuracy)
study = optuna.create_study(direction='maximize',
                             pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=5))

# Run the optimization
study.optimize(objective, n_trials = 30)

# Print the best hyperparameters found
print("Best hyperparameters: ", study.best_params)

# Print the best accuracy found
print("Best accuracy: ", study.best_value)

# Plot the visualization
optuna.visualization.plot_param_importances(study).show()

# Run more trials
# study.optimize(objective, n_trials=20)

[I 2025-11-02 20:06:17,492] A new study created in memory with name: no-name-ea706092-424e-4c8d-b485-8018e86bfd02



Trial 0 | lr=0.000121 | optimizer=AdamW | batch=128 | hidden=177
Epoch [ 1/7 ] Train Acc: 0.5470
Epoch [ 2/7 ] Train Acc: 0.7730
Epoch [ 3/7 ] Train Acc: 0.8745
Epoch [ 4/7 ] Train Acc: 0.9222
Epoch [ 5/7 ] Train Acc: 0.9418
Epoch [ 6/7 ] Train Acc: 0.9556
Epoch [ 7/7 ] Train Acc: 0.9645


[I 2025-11-02 20:35:44,382] Trial 0 finished with value: 0.31391830559757944 and parameters: {'lr': 0.000120894002107495, 'optimizer': 'AdamW', 'weight_decay': 0.0005898044823550142, 'hidden_size': 177, 'batch_size': 128, 'num_epochs': 7, 'fc_drop_rate': 0.41887942963349045, 'cnn_drop_rate': 0.05499558346634281}. Best is trial 0 with value: 0.31391830559757944.


Validation Acc: 0.3139


Trial 1 | lr=0.038477 | optimizer=RMSprop | batch=32 | hidden=130
Epoch [ 1/10 ] Train Acc: 0.1264
Epoch [ 2/10 ] Train Acc: 0.1282
Epoch [ 3/10 ] Train Acc: 0.1247
Epoch [ 4/10 ] Train Acc: 0.1278
Epoch [ 5/10 ] Train Acc: 0.1264
Epoch [ 6/10 ] Train Acc: 0.1242
Epoch [ 7/10 ] Train Acc: 0.1260
Epoch [ 8/10 ] Train Acc: 0.1232
Epoch [ 9/10 ] Train Acc: 0.1246
Epoch [10/10 ] Train Acc: 0.1239


[I 2025-11-02 21:16:44,771] Trial 1 finished with value: 0.12427058569267344 and parameters: {'lr': 0.038477129729780964, 'optimizer': 'RMSprop', 'momentum': 0.7945351558191214, 'weight_decay': 0.0029810145884405772, 'hidden_size': 130, 'batch_size': 32, 'num_epochs': 10, 'fc_drop_rate': 0.21933425376416027, 'cnn_drop_rate': 0.013911218331316}. Best is trial 0 with value: 0.31391830559757944.


Validation Acc: 0.1243


Trial 2 | lr=0.000010 | optimizer=Adadelta | batch=16 | hidden=215
Epoch [ 1/5 ] Train Acc: 0.1388
Epoch [ 2/5 ] Train Acc: 0.1585
Epoch [ 3/5 ] Train Acc: 0.1781
Epoch [ 4/5 ] Train Acc: 0.1925
Epoch [ 5/5 ] Train Acc: 0.2018


[I 2025-11-02 21:39:58,048] Trial 2 finished with value: 0.17397881996974282 and parameters: {'lr': 1.0054766400230084e-05, 'optimizer': 'Adadelta', 'weight_decay': 0.0085689481939001, 'hidden_size': 215, 'batch_size': 16, 'num_epochs': 5, 'fc_drop_rate': 0.3515580985452771, 'cnn_drop_rate': 0.11695352796637568}. Best is trial 0 with value: 0.31391830559757944.


Validation Acc: 0.1740


Trial 3 | lr=0.001403 | optimizer=AdamW | batch=128 | hidden=123
Epoch [ 1/8 ] Train Acc: 0.4877
Epoch [ 2/8 ] Train Acc: 0.7839
Epoch [ 3/8 ] Train Acc: 0.8961


[W 2025-11-02 21:53:35,898] Trial 3 failed with parameters: {'lr': 0.0014033867869529925, 'optimizer': 'AdamW', 'weight_decay': 0.005077267609769627, 'hidden_size': 123, 'batch_size': 128, 'num_epochs': 8, 'fc_drop_rate': 0.2513304325786173, 'cnn_drop_rate': 0.1759311401506968} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/optuna/study/_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipython-input-1905089268.py", line 137, in objective
    for images, metadata, labels in train_loader:
                                    ^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 734, in __next__
    data = self._next_data()
           ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 790, in _next_data
    data = self._dataset_fetcher.f

KeyboardInterrupt: 

#Sources:
###Hyperparameter Tuning with Optuna:
https://medium.com/@taeefnajib/hyperparameter-tuning-using-optuna-c46d7b29a3e

https://optuna.org/#code_examples
###Multi-Modal ML Models
https://www.nature.com/articles/s41598-025-14901-4
https://www.reddit.com/r/MachineLearning/comments/nziumg/combining_images_and_other_numeric_features_in_a/
https://pyimagesearch.com/2019/02/04/keras-multiple-inputs-and-mixed-data/

###Next Models to test:
VideoGasNet:
https://www.sciencedirect.com/science/article/pii/S0360544221017643

GasVit: https://www.sciencedirect.com/science/article/pii/S1568494623011560?via%3Dihub#sec3